In [4]:
!pip install krwordrank

In [14]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 1) CSV 불러오기
df = pd.read_csv("../mini_project1/preprocessed_data/4분기_일자별_전처리1.csv")

target_col = "title"   # 필요하면 "제목"으로 바꿔도 됨
df[target_col] = df[target_col].fillna("")

# 2) 불용어 - 긍정/부정 관련
stopwords = {
    # 1. 시간/시기/접속사 관련 (Time/Conjunction)
    '장중', '만에', '연속', '마감', '출발', 
    '이날', '오전', '오후', '하루', '주간', '월간', '올해', '내년', '작년', '지난해',
    '앞두고', '가운데', '속', '종합',# <--- 추가된 단어들

    # 2. 상태/등락/감성 관련 (State/Fluctuation) -> 감성 사전으로 이동할 것들
    '상승', '급등', '급락', '반등', '회복', '돌파', '동반', '경신', '확대', '용인',
    '강보합', '약보합', '약세', '강세', '보합', '혼조', '하락', '갱신', '지속', '쇼크',
    '붕괴', '추락', '급감', '폭락', # <--- 추가된 단어들 ('붕괴' 포함)

    # 3. 수량/정도/행동 (Quantity/Action)
    '최대', '최고치', '사상', '기대', '매수', '전망', '우려', '부진', '개월', '조원', '기대감', '연고점','최고',

    # 4. 동사
    '사자', '코스닥도', '전환', '팔자', '매수에',

    # 5. 기타
    'sk', '분기', '하이닉스', '만전자'

}

# 3) 토크나이저: 숫자 제거, 한 글자 제거, 불용어 제거
def tokenize(text: str):
    # 한글/영문만 남기고 나머지 제거
    text = re.sub(r"[^가-힣A-Za-z ]", " ", text)
    text = text.lower()
    tokens = text.split()

    clean_tokens = []
    for w in tokens:
        # 한 글자 제거
        if len(w) < 2:
            continue
        # 숫자 포함 단어 제거
        if re.search(r"[0-9]", w):
            continue
        # 불용어 제거
        if w in stopwords:
            continue
        # 'sk하이닉스'가 포함된 형태들 제거 (예: sk하이닉스가, sk하이닉스는)
        if "sk하이닉스" in w:
            continue

        clean_tokens.append(w)

    return clean_tokens

# 4) TF-IDF 벡터화
vectorizer = TfidfVectorizer(
    tokenizer=tokenize,
    token_pattern=None  # 우리가 직접 tokenizer 쓸 거라 None
)

docs = df[target_col].astype(str).tolist() # 기사들 추출
tfidf_matrix = vectorizer.fit_transform(docs)
feature_names = vectorizer.get_feature_names_out()

n_docs = tfidf_matrix.shape[0]

# 5) 문서비율(df) 계산 (해당 단어가 등장한 문서 비율)
doc_freq = (tfidf_matrix > 0).sum(axis=0).A.ravel()
doc_ratio = doc_freq / n_docs

# df 필터: 1% 이상, 50% 이하만 사용 (너무 흔하거나 너무 희귀한 단어 제거)
min_df_ratio = 0.01   # 1%
max_df_ratio = 0.50   # 50%

valid_mask = (doc_ratio >= min_df_ratio) & (doc_ratio <= max_df_ratio)
valid_indices = np.where(valid_mask)[0]

valid_features = feature_names[valid_indices] # 1차적으로 뽑힌 단어들
# print('필터링 되기 전:',valid_features)
valid_tfidf = tfidf_matrix[:, valid_indices]

# 6) 각 단어의 평균 TF-IDF 점수
mean_tfidf = valid_tfidf.mean(axis=0).A.ravel()
word_scores = list(zip(valid_features, mean_tfidf)) # 단어 - 가중치 점수 매핑된 거 
print(len(word_scores))
# print('단어점수:',word_scores)

# 7) 평균 TF-IDF 기준 Top 50 단어
top30 = sorted(word_scores, key=lambda x: x[1], reverse=True)[:30] # 매핑된 최종 단어들 (여기에서 첫 값들만 추출)
# print(top30)

print("=== 필터링 후 Top 30 키워드 ===")
for w, _ in top30:# 첫 단어들만 추출 
    print(w)

# Top50 단어 목록만 따로 추출
top_words = [w for w, _ in top30] # 나와야하는 단어들 -> list
# print(type(top_words))

# valid_features 에서 top30 단어의 인덱스 찾기
word_to_idx = {w: i for i, w in enumerate(valid_features)}
top_indices = [word_to_idx[w] for w in top_words]

# 각 문서별 top30 단어 TF-IDF 매트릭스 (n_docs x 50)
top_tfidf = valid_tfidf[:, top_indices].toarray()

# 8) 문서별 최대값을 1(=100%)로 보이도록 정규화
#   - 각 제목에서 가장 높은 TF-IDF = 1.0 → 100%
row_max = top_tfidf.max(axis=1, keepdims=True)
row_max[row_max == 0] = 1  # 전부 0인 경우 0으로 나누기 방지

top_tfidf_pct = (top_tfidf / row_max)  # 이 줄은 여기까지만 쓰고 아래는 새로 추가

# 9) TOP50 키워드별로 "제목에서 등장한 빈도 비율(0~1)" 계산
#    doc_ratio: 전체 vocabulary 기준 (단어가 등장한 문서 수 / 전체 문서 수)
#    valid_indices: 그중 df 1%~50% 필터 통과한 단어 인덱스
doc_ratio_valid = doc_ratio[valid_indices]  # valid_features와 같은 순서

# top_words 각각에 대해 빈도 비율 추출
top_freq_ratio = np.array([doc_ratio_valid[word_to_idx[w]] for w in top_words])

# 10) CSV 저장: 컬럼 2개 (keyword, ratio)
result_df = pd.DataFrame({
    "keyword": top_words,        # TOP50 키워드
    "ratio": top_freq_ratio      # 각 키워드가 등장한 제목 비율 (0~1, 1이 100%)
})

result_df.to_csv("../mini_project1/preprocessed_data/4분기_키워드_top30.csv",
                 index=False,
                 encoding="utf-8-sig")

print("출력 완료")












53
=== 필터링 후 Top 30 키워드 ===
코스피
ai
반도체
삼성전자
외국인
기관
엔비디아
hbm
삼성
증시
외인
만닉스
특징주
삼전
실적
메모리
환율
만원
마감시황
코스닥
최태원
속보
개장시황
투자
훈풍에
글로벌
시대
주가
시총
거품론
출력 완료
